In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
import numpy as np
from numpy import random
from time import time

In [2]:
#load data (make sure you have downloaded database.sqlite)
with sqlite3.connect('../soccer/database.sqlite') as con:
    Player_Attributes = pd.read_sql_query("SELECT * from Player_Attributes", con)
Player_Attributes.shape

(183978, 42)

In [3]:
#select relevant fields
Player_Attributes.dropna(inplace=True)
Player_Attributes.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date'], axis = 1, inplace = True)
overall_rating = Player_Attributes['overall_rating']
features = Player_Attributes.drop('overall_rating', axis = 1)
features.head()

,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [4]:
# Use LabelEncoder to convert categorical data field into numerical data field
'''from sklearn import preprocessing

le_sex = preprocessing.LabelEncoder()

#to convert into numbers

features.preferred_foot = le_sex.fit_transform(features.preferred_foot)
features.attacking_work_rate = le_sex.fit_transform(features.attacking_work_rate)
features.defensive_work_rate = le_sex.fit_transform(features.defensive_work_rate)
features.head()

# to convert back
# train.Sex = le_sex.inverse_transform(train.Sex)'''

'from sklearn import preprocessing\n\nle_sex = preprocessing.LabelEncoder()\n\n#to convert into numbers\n\nfeatures.preferred_foot = le_sex.fit_transform(features.preferred_foot)\nfeatures.attacking_work_rate = le_sex.fit_transform(features.attacking_work_rate)\nfeatures.defensive_work_rate = le_sex.fit_transform(features.defensive_work_rate)\nfeatures.head()\n\n# to convert back\n# train.Sex = le_sex.inverse_transform(train.Sex)'

In [5]:
# Use pandas get_dummies to convert categorical value into numerical
features = pd.get_dummies(features)

In [6]:
# Feature scaling using MinMaxScaler
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled_features = min_max_scaler.fit_transform(features)

In [7]:
'''#b Let's use Standard scaler now to scale the data
from sklearn import preprocessing
std_scaler = preprocessing.StandardScaler()
scaled_features = std_scaler.fit_transform(features)'''

"#b Let's use Standard scaler now to scale the data\nfrom sklearn import preprocessing\nstd_scaler = preprocessing.StandardScaler()\nscaled_features = std_scaler.fit_transform(features)"

In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 6)
pca_features = pca.fit_transform(scaled_features)

In [9]:
'''# Feature selection using sklearn SelectKBest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
feature_reg = SelectKBest(f_regression, k=6)
X_new = feature_reg.fit_transform(scaled_features, overall_rating)'''

'# Feature selection using sklearn SelectKBest\nfrom sklearn.feature_selection import SelectKBest\nfrom sklearn.feature_selection import f_regression\nfeature_reg = SelectKBest(f_regression, k=6)\nX_new = feature_reg.fit_transform(scaled_features, overall_rating)'

In [15]:
# Train and predict model on Decision tree,on SGD regressor and K-Nearest Neighbors
from sklearn import tree
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from time import time

reg1 = tree.DecisionTreeClassifier()
reg2 = linear_model.SGDRegressor()
reg3 = KNeighborsRegressor(n_neighbors=2)
reg4 = MLPRegressor(random_state=1, max_iter=500)


regs = {reg1:"DecisionTree", reg2:"SGDRegressor", reg3:"KNeighbors", reg4:"NeuralNetwork"}

for key in regs:
    t0 = time()
    X_train, X_test, y_train, y_test = train_test_split(scaled_features, overall_rating, test_size=0.25, random_state=0)

    print ("--------------------")
    print (regs[key])
    print ("--------------------")
        
    t1 = time()
    key.fit(X_train, y_train)
    print ("Time taken to train the model: {}".format(time()-t1))

    t2 = time()
    pred_test = key.predict(X_test)
    pred_train = key.predict(X_train)
    print ("Time taken to predict the model: {}".format(time()-t2))

    t3 = time()
    print ("r2 score of this model on testing set is: {}".format(r2_score(y_test, pred_test)))
    print ("r2 score of this model on training set is: {}".format(r2_score(y_train, pred_train)))

--------------------
DecisionTree
--------------------
Time taken to train the model: 6.161545753479004
Time taken to predict the model: 0.14085721969604492
r2 score of this model on testing set is: 0.9484164594945248
r2 score of this model on training set is: 0.9988644730721816
--------------------
SGDRegressor
--------------------
Time taken to train the model: 2.4566903114318848
Time taken to predict the model: 0.016827106475830078
r2 score of this model on testing set is: 0.8426632081245964
r2 score of this model on training set is: 0.8443254385919994
--------------------
KNeighbors
--------------------
Time taken to train the model: 13.631422519683838
Time taken to predict the model: 703.6021955013275
r2 score of this model on testing set is: 0.9557633414780853
r2 score of this model on training set is: 0.9891838192633062
--------------------
NeuralNetwork
--------------------
Time taken to train the model: 833.5611526966095
Time taken to predict the model: 5.09360671043396
r2 sco

In [ ]:
from sklearn.datasets import load_iris
X_train, y_train = load_iris(return_X_y=True)
tree.plot_tree(reg1) 

In [12]:
# Use GridSearch to tune the model
def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    from sklearn.cross_validation import ShuffleSplit
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)

    # TODO: Create a decision tree regressor object
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.svm import SVC
    from sklearn import linear_model
    
    
    regressor1 = DecisionTreeRegressor()
    regressor2 = SVC()
    regressor3 = linear_model.SGDRegressor()

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    tree_params = {'max_depth' : [3, 6, 9, 20, 100], 'min_samples_split':[2, 3, 4, 5]}
    svm_params = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
    sgd_params = {'loss':['squared_loss', 'huber'], 'penalty': ['none', 'l2', 'l1', 'elasticnet'], 'n_iter':[10, 75, 100, 500]}
    
    
    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    from sklearn.metrics import make_scorer
    scoring_fnc = make_scorer(performance_metric)

    # TODO: Create the grid search object
    from sklearn.grid_search import GridSearchCV
    
    # Updated cv_sets and scoring parameter
    grid = GridSearchCV(regressor3, sgd_params, scoring = scoring_fnc, cv = cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

def performance_metric(y_true, y_predict):
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    from sklearn.metrics import r2_score
    
    score = r2_score(y_true, y_predict)
    # Return the score
    return score

In [15]:
'''from time import time
t0 = time()
grid_reg = fit_model(pca_features, overall_rating)
print (grid_reg.score)
# grid_pred = grid_reg()
print ("Time taken to train and predict using GridSearch: {}".format(time() - t0))
print ("Best parameters are: {}".format(grid_reg.get_params()))'''

'from time import time\nt0 = time()\ngrid_reg = fit_model(pca_features, overall_rating)\nprint (grid_reg.score)\n# grid_pred = grid_reg()\nprint ("Time taken to train and predict using GridSearch: {}".format(time() - t0))\nprint ("Best parameters are: {}".format(grid_reg.get_params()))'